In [1]:
import numpy as np
import pandas as pd
import re
import time
from datasketch import MinHash, MinHashLSHForest
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import warnings
import model_report as mr
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
file_info = "C:\\Users\\blgai\\OneDrive\\Documents\\School\\SMU\\Courses\\Fall 2021\\Capstone A\Data\\cleaned_chunked_v2.csv"
df = pd.read_csv(file_info, sep='\t')

In [3]:
#balance classes
g = df.groupby('category')
df = pd.DataFrame(g.apply(lambda x: x.sample(g.size().min()).reset_index(drop=True)))

In [4]:
df.head()

url  code category language  \
category                                                                
bu       0                 *://DECENTLITES.IN   200       bu       en   
         1                          *://WV.LY   200       bu       en   
         2          *://AIMRECYCLINGGROUP.COM   200       bu       en   
         3            *://danforthfilters.com   200       bu       en   
         4  *://WWW.SUPERIORSEALANDPAVING.COM   200       bu       en   

                                                        title  \
category                                                        
bu       0                                DECENT LITES - Home   
         1                      Bitly - The Power of the Link   
         2                    American Iron & Metal - Accueil   
         3  Air Filters & Air Purifiers - Discount Air Fil...   
         4                             Superior Seal & Paving   

                                                      summary  \
category                                                        
bu       0  one of the widely acclaimed companies engaged ...   
         1  Bitly Brand Tools: Own, understand and activat...   
         2  AIM is a Canadian company that recuperates, we...   
         3  Air Filters & Air Purifiers - Danforth Air Fil...   
         4  Superior Seal & Paving, Inc is a privately own...   

                                                    key_words  \
category                                                        
bu       0  Home improvement, DECENT LITES, AGRA ROAD, FIR...   
         1  Bitly Brand Tools: Own, understand and activat...   
         2  AIM Recycling USA, AIM Recycling, AIM USA, AIM...   
         3  Air Filters Air Purifiers Danforth Air Filtrat...   
         4  Superior Seal & Paving, Paving in Syracuse, Se...   

                                                      content  \
category                                                        
bu       0  DECENT LITES DECENT LITES AGRA ROAD FIROZABAD ...   
         1  Bitly Product Tour Resources Partners Sign In ...   
         2  Login Accueil Profil Corporatif La direction H...   
         3  Home Air Filters Housings Air Cleaners Air Pur...   
         4  RSS Search Home About Us Paving Sealing Meet o...   

                                                     combined  \
category                                                        
bu       0  DECENT LITES - Home one of the widely acclaime...   
         1  Bitly - The Power of the Link Bitly Brand Tool...   
         2  American Iron & Metal - Accueil AIM is a Canad...   
         3  Air Filters & Air Purifiers - Discount Air Fil...   
         4  Superior Seal & Paving Superior Seal & Paving,...   

                                                        nouns  \
category                                                        
bu       0  [('LITES', 7, 12, 'NOUN'), ('Home', 15, 19, 'N...   
         1  [('Power', 12, 17, 'PROPN'), ('Link', 25, 29, ...   
         2  [('American', 0, 8, 'PROPN'), ('Iron', 9, 13, ...   
         3  [('Air', 0, 3, 'PROPN'), ('Filters', 4, 11, 'P...   
         4  [('Superior', 0, 8, 'PROPN'), ('Seal', 9, 13, ...   

                                                    compounds  \
category                                                        
bu       0  [('LITES -', 7, 14, 'COMPOUND'), ('DECENTLITES...   
         1  [('Link Bitly', 25, 35, 'COMPOUND'), ('Brand T...   
         2  [('Accueil AIM', 24, 35, 'COMPOUND'), ('scrap ...   
         3  [('Air Filters', 0, 11, 'COMPOUND'), ('Air Pur...   
         4  [('Superior Seal', 0, 13, 'COMPOUND'), ('Super...   

                                                   comp_nouns  \
category                                                        
bu       0  {'Xmas', 'stains', 'paragon', 'cracking', 'Chr...   
         1  {'Links Shortened Worldwide Shorten', 'Tech', ...   
         2  {'scrap', 'auto salvage', 'alliages', 'Cuivre'...   
         3  {

In [5]:
df.groupby(df.category).size()

category
bu    1595
dr    1595
ed    1595
mk    1595
os    1595
sp    1595
sx    1595
dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
#get 10% holdout set
train_corpus, holdout_corpus, train_label, holdout_label = train_test_split(df['title'], 
                                                                           df['category'],
                                                                           test_size=.10,random_state=1234)

#get 80% train and 20% test sets
train_corpus, test_corpus, train_label, test_label = train_test_split(train_corpus,
                                                                      train_label,
                                                                      test_size=.2225,random_state=1234) #.9 * .2225 = .20

train_corpus.shape, test_corpus.shape, holdout_corpus.shape

((7812,), (2236,), (1117,))

In [7]:
df_train = pd.DataFrame(zip(train_corpus,train_label),columns=["title","label"])
df_test = pd.DataFrame(zip(test_corpus,test_label),columns=["title","label"])
df_holdout = pd.DataFrame(zip(holdout_corpus,holdout_label),columns=["title","label"])

In [8]:
df_train.head()

,title,label
0,BLUE CORP - The Innovative Thinking,bu
1,Modded Controllers for Xbox and PlayStation | ...,os
2,Niall Scully Photography | Wedding Photographe...,mk
3,Increase Sales Force Effectiveness | Precallpr...,bu
4,Project Management in Practice: Practical Plan...,bu


In [9]:
#Number of Permutations
permutations = 256


In [10]:
def preprocess(text):
    text = re.sub(r'[^\w\s]','',text)
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

In [11]:
def get_forest(data,perms):
    start_time = time.time()
    minhash = []
    for text in data['title']:
        tokens = preprocess(text)
        m = MinHash(num_perm=perms)
        for s in tokens:
            m.update(s.encode('utf8'))
        minhash.append(m)
        
    forest = MinHashLSHForest(num_perm=perms)
    
    for i,m in enumerate(minhash):
        forest.add(i,m)
    
    forest.index()
    
    print('It took %s seconds to build forest.' %(time.time()-start_time))
    
    return forest

In [12]:
def get_cosine(list1,list2):
    from collections import Counter

    # count word occurrences
    a_vals = Counter(list1)
    b_vals = Counter(list2)

    # convert to word-vectors
    words  = list(a_vals.keys() | b_vals.keys())
    a_vect = [a_vals.get(word, 0) for word in words]        
    b_vect = [b_vals.get(word, 0) for word in words]        

    # find cosine
    len_a  = sum(av*av for av in a_vect) ** 0.5             
    len_b  = sum(bv*bv for bv in b_vect) ** 0.5             
    dot    = sum(av*bv for av,bv in zip(a_vect, b_vect))    
    cosine = dot / (len_a * len_b)                          
    
    return cosine

In [13]:
def get_similars(test_item, database, perms, num_results, forest):
        
    tokens = preprocess(test_item)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
    
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None #if query is empty, return none
    
    #label = database.iloc[idx_array]['label']
    #text = database.iloc[idx_array]['clean_comp_nouns']
    df_results = pd.DataFrame(database.iloc[idx_array][['title','label']])
    
    return df_results

In [14]:
def predict(test_item, database, perms, num_results, forest):
    df_pred = get_similars(test_item,df_train,permutations,num_similars,forest)
    df_pred['cos_dist'] = df_pred.apply(lambda x: get_cosine(preprocess(x['title']),text),axis=1)
    
    return df_pred.sort_values('cos_dist',ascending=False)
    

In [15]:
def predict_one(test_item, database, perms, num_results, forest):
    df_pred_one = predict(test_item, database, perms, num_results, forest)
    #get most similar item based on cosine distance
    cat = None
    try:
        cat = df_pred_one.iloc[1, 1]
    except:
        cat = 'ed'
    
    return cat
    

In [16]:
def predict_mod(test_item, database, perms, num_results, forest):
    label = None
    try:
        df_pred_mod = get_similars(test_item,df_train,permutations,num_similars,forest)
        label = df_pred_mod['label'].value_counts()[:1].index.tolist()[0]
    except:
        return None
    return label

In [17]:
#build lsh forest using training data
forest = get_forest(df_train,permutations)

It took 16.064402103424072 seconds to build forest.


In [18]:
#Number of similars to return
num_similars = 50
test_item = 5
text = df_test['title'][test_item]
#query the forest
df_predict = predict(text,df_train,permutations,num_similars,forest)
print('actual label: ',df_test['label'][test_item])
print('mod similar: ',predict_mod(text,df_train,permutations,num_similars,forest))
print('closest similar: ',predict_one(text,df_train,permutations,num_similars,forest))
print('All similars: ',predict(text,df_train,permutations,num_similars,forest))
text

actual label:  ed
mod similar:  bu
closest similar:  sp
All similars:                                                    title label  cos_dist
5248  hamsterclicks.com : Welcome To hamsterclicks.com!    mk       0.0
3679                                  WELCOME RACE FANS    sp       0.0
2370                   Welcome to Magazine Rewards Plus    mk       0.0
707                                            WELCOME!    ed       0.0
4036             :: Welcome to Bulkbag Manufacturers ::    bu       0.0
4167                Welcome to Ugborough Primary School    ed       0.0
7118                Welcome to the HENNEPDESK Community    dr       0.0
2256                     Welcome to The Power Kite Site    sp       0.0
4437       Welcome to The Linde Group | The Linde Group    bu       0.0
5210                                 Isprojects Welcome    mk       0.0
5085                         Welcome To SpankingDVD.com    sx       0.0
4317                          Welcome to COCKSUCKERS.TV    sx    

'Welcome to TallTrees Montessori-inspired School!'

In [20]:
df_results = pd.DataFrame(list(zip(df_test['title'],df_test['label'])),columns=['terms','category'])
df_results['lsh_predict'] = df_results.apply(lambda x: predict_mod(x['terms'],df_train,permutations,num_similars,forest),axis=1)
df_results['match'] = np.where(df_results['category']==df_results['lsh_predict'],1,0)

In [21]:
df_results[df_results['lsh_predict'].isna()]

,terms,category,lsh_predict,match
3,Saeco Arad > GERA COM > Aparate Cafea - Espresso,bu,None,0
9,EduQuality EduQuality,ed,None,0
18,Cantiere Nautico - Cranchi,mk,None,0
20,MavPuck.com,sp,None,0
39,ShopPal,os,None,0
...,...,...,...,...
2146,http://heartwooddesign.co/millwork/solid-wood-...,mk,None,0
2147,หน้าหลัก | Mainpage ข้อมูล ข่าวสาร งานวิจัย,ed,None,0
2160,Verachan,os,None,0
2190,Skoleporten Rugkobbelskolen Skoleporten - Rugk...,ed,None,0


In [22]:
rpt = mr.generate_report(df_results)
rpt

{'totalSamples': 2236,
 'overallAccuracy': 0.5380143112701252,
 'byCategory': {'dr': {'totalSamples': 331, 'accuracy': 0.7280966767371602},
  'sx': {'totalSamples': 332, 'accuracy': 0.713855421686747},
  'ed': {'totalSamples': 330, 'accuracy': 0.5151515151515151},
  'sp': {'totalSamples': 318, 'accuracy': 0.5031446540880503},
  'os': {'totalSamples': 314, 'accuracy': 0.45222929936305734},
  'bu': {'totalSamples': 311, 'accuracy': 0.42443729903536975},
  'mk': {'totalSamples': 300, 'accuracy': 0.4033333333333333}}}

In [23]:
from sklearn.metrics import confusion_matrix
cnf_matrix = confusion_matrix(df_results['category'],df_results['lsh_predict'],labels=['os','ed','dr','sp','mk','sx','bu'])
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
mr.plot_confusion_matrix(cnf_matrix, classes=['os','ed','dr','sp','mk','sx','bu'],
                      title='Confusion matrix, LSH Forest')

TypeError: '<' not supported between instances of 'NoneType' and 'str'

In [ ]:
#Number of similars to return
num_similars = 15
test_item = 9
text = df_test['title'][test_item]
df_my_test_mod = predict(text,df_train,permutations,num_similars,forest)
print('actual label: ',df_test['label'][test_item])
df_my_test_mod['label'].value_counts()[:1].index.tolist()[0]